# Advent of Code - Day 18

I'm sure there must be a clever way of viewing this to trick the system into redoing the operator precedence, but I can't think of it at the moment.

Wasted quite a lot of time on this earlier trying to use the challenge as an excuse to learn a new parser. `pyparsing` looked good. It turns out to have *by far* the *worst* documentation I have seen for any python module. Including the very early incarnations of pandas.

## Part 1

In [1]:
import math
import re

import functools as ft
import operator

Let's find the deepest nested bracketed expression and evaluate it. Then replace the expression in the string.

Start with evaluating an expression if it has no brackets (and no bothering to check well formedness, I'm afraid):

In [2]:
# If the expression is only made up of numbers and '+', evaluate:

def eval_flat_ltor(str_in):
    
    expr_ls=re.findall('\d+|\+|\*', str_in)
    
    out = int(expr_ls.pop(0))

    while expr_ls:
        fn=expr_ls.pop(0)
        arg=int(expr_ls.pop(0))
        if fn=='+':
            out+=arg
        elif fn=='*':
            out*=arg
            
    return out

eval_flat_ltor('4 +2 * 3 +1')

19

In [3]:
def eval_expr(str_in):
    m=re.search('\(([^(]*?)\)', str_in) # find bracketed expr
    if m:
        (start, end)=m.span()
   
        return eval_expr(' '.join([str_in[:start],
                                   str(eval_flat_ltor(m.group(1))),
                                   str_in[end:]]))
    else:
        return eval_flat_ltor(str_in)

In [4]:
assert eval_expr("2 * 3 + (4 * 5)") == 26
assert eval_expr("5 + (8 * 3 + 9 + 3 * 4 * 3)") == 437
assert eval_expr("5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))") == 12240
assert eval_expr("((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2") == 13632

In [5]:
def day18_part1(file_in):
    
    return sum([eval_expr(expr)
                for expr in open(file_in).read().splitlines()])

In [6]:
day18_part1('data/day18_input')

45840336521334

## Part 2

Same sort of thing again, but this time, use a flat evaluator that respects the inverted precedence.

In [7]:
import re
import functools as ft

For some reason, my `eval` doesn't seem to be working, so do all the evaluations by hand:

In [8]:
# If the expression is only made up of numbers and '+', evaluate:

def eval_flat_sum_prec(str_in):

    if re.match('^[+\d\s]*$', str_in):
        return sum([int(x.strip()) for x in str_in.split('+')])

    if re.match('^[*\d\s]*$', str_in):
        return ft.reduce(operator.mul,
                         [int(x.strip()) for x in str_in.split('*')])
    
    # Match a pair and replace with the evaluation:
    
    m=re.search('(\d+)\s*\+\s*(\d+)', str_in)
    if m:
        (start, end)=m.span()
        return eval_flat_sum_prec(' '.join([str_in[:start],
                         str(int(m.group(1)) + int(m.group(2))),
                         str_in[end:]]))

In [9]:
eval_flat_sum_prec('2*3+4*5+2')

98

In [10]:
def eval_expr_pt2(str_in):
    m=re.search('\(([^(]*?)\)', str_in)
    if m:
        (start, end)=m.span()
   
        return eval_expr_pt2(' '.join([str_in[:start],
                                   str(eval_flat_sum_prec(m.group(1))),
                                   str_in[end:]]))
    else:
        return eval_flat_sum_prec(str_in)

In [11]:
def day18_part2(str_in):
    return eval_expr_pt2(str_in)

In [12]:
assert day18_part2('1 + (2 * 3) + (4 * (5 + 6))') == 51
assert day18_part2('2 * 3 + (4 * 5)') == 46
assert day18_part2('5 + (8 * 3 + 9 + 3 * 4 * 3)') == 1445
assert day18_part2('5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))') == 669060
assert day18_part2('((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2') == 23340


In [13]:
sum([day18_part2(nl) for nl in open('data/day18_input').readlines()])

328920644404583

Done!